# In Depth A/B Testing - Lab

## Introduction

In this lab, you'll explore a survey from Kaggle regarding budding data scientists. With this, you'll form some initial hypotheses, and test them using the tools you've acquired to date. 

## Objectives

You will be able to:
* Conduct t-tests and an ANOVA on a real-world dataset and interpret the results

## Load the Dataset and Perform a Brief Exploration

The data is stored in a file called **multipleChoiceResponses_cleaned.csv**. Feel free to check out the original dataset referenced at the bottom of this lab, although this cleaned version will undoubtedly be easier to work with. Additionally, meta-data regarding the questions is stored in a file name **schema.csv**. Load in the data itself as a Pandas DataFrame, and take a moment to briefly get acquainted with it.

> Note: If you can't get the file to load properly, try changing the encoding format as in `encoding='latin1'`

In [1]:
import pandas as pd
%run 'flatiron_stats.py'

In [2]:
df = pd.read_csv('multipleChoiceResponses_cleaned.csv', encoding='latin1')
df

/Users/freethrall/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compile

,GenderSelect,Country,Age,EmploymentStatus,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,...,JobFactorTitle,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity,exchangeRate,AdjustedCompensation
0,"Non-binary, genderqueer, or gender non-conforming",NaN,NaN,Employed full-time,NaN,NaN,Yes,NaN,DBA/Database Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Female,United States,30.0,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Somewhat important,NaN,NaN,NaN,NaN
2,Male,Canada,28.0,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,NaN,...,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,NaN,NaN
3,Male,United States,56.0,"Independent contractor, freelancer, or self-em...",NaN,NaN,Yes,NaN,Operations Research Practitioner,Poorly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,250000.0
4,Male,Taiwan,38.0,Employed full-time,NaN,NaN,Yes,NaN,Computer Scientist,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26389,Female,Other,24.0,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26390,Male,Indonesia,25.0,Employed full-time,NaN,NaN,Yes,NaN,Programmer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000076,NaN
26391,Female,Taiwan,25.0,Employed part-time,NaN,NaN,No,Yes,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26392,Female,Singapore,16.0,I prefer not to say,Yes,"Yes, but data science is a small part of what ...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Wages and Education

You've been asked to determine whether education is impactful to salary. Develop a hypothesis test to compare the salaries of those with Master's degrees to those with Bachelor's degrees. Are the two statistically different according to your results?

> Note: The relevant features are stored in the 'FormalEducation' and 'AdjustedCompensation' features.

You may import the functions stored in the `flatiron_stats.py` file to help perform your hypothesis tests. It contains the stats functions that you previously coded: `welch_t(a,b)`, `welch_df(a, b)`, and `p_value(a, b, two_sided=False)`. 

Note that `scipy.stats.ttest_ind(a, b, equal_var=False)` performs a two-sided Welch's t-test and that p-values derived from two-sided tests are two times the p-values derived from one-sided tests. See the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) for more information.    

In [3]:
# H0: Education is not impactful to salary. People with Master's degrees make the same
# salary as those with Bachelor's degrees

# HA: Education is impactful to salary: People with Master's degrees have a higher salary
# than those with a Bachelor's degree

In [4]:
df.FormalEducation.unique()

array(["Bachelor's degree", "Master's degree", 'Doctoral degree', nan,
       "Some college/university study without earning a bachelor's degree",
       'I did not complete any formal education past high school',
       'Professional degree', 'I prefer not to answer'], dtype=object)

In [5]:
# relevent columns --> AdjustedCompensation, FormalEducation
masters_df = df[df.FormalEducation == "Master's degree"]['AdjustedCompensation']
clean_masters_df = masters_df.dropna()
clean_masters_df

3        250000.000
14        36634.400
27        53352.000
31        35419.104
37        80000.000
            ...    
26148     54670.000
26159         1.000
26180     65770.430
26255     89686.950
26378     10000.000
Name: AdjustedCompensation, Length: 1990, dtype: float64

In [6]:
bachelors_df = df[df.FormalEducation == "Bachelor's degree"]['AdjustedCompensation']
clean_bachelors_df = bachelors_df.dropna()
clean_bachelors_df

8         64184.800
9         20882.400
11         1483.900
21        20000.000
25        10858.848
            ...    
26031     39050.000
26072     31878.000
26101      3336.000
26185     50000.000
26195    100449.384
Name: AdjustedCompensation, Length: 1107, dtype: float64

In [7]:
p = p_value_welch_ttest(clean_bachelors_df, clean_masters_df, two_sided=False)
print(f'Given that our calculated p-value is {round(p,3)}, which is greater than our alpha of 0.05')
print(f'We cannot reject the Null Hypothesis and cannot show that education has a positive impact')
print('on salary')

Given that our calculated p-value is 0.331, which is greater than our alpha of 0.05
We cannot reject the Null Hypothesis and cannot show that education has a positive impact
on salary


## Wages and Education II

Now perform a similar statistical test comparing the AdjustedCompensation of those with Bachelor's degrees and those with Doctorates. If you haven't already, be sure to explore the distribution of the AdjustedCompensation feature for any anomalies. 

In [8]:
doc_df = df[df.FormalEducation == "Doctoral degree"]['AdjustedCompensation']
clean_doc_df = doc_df.dropna()
clean_doc_df

22       100000.000
32       172144.440
34       133000.000
61        15000.000
72        43049.736
            ...    
25875     71749.560
25966     12000.000
26012    123553.200
26038    170000.000
26203    200000.000
Name: AdjustedCompensation, Length: 967, dtype: float64

In [9]:
clean_doc_df.describe()

count    9.670000e+02
mean     2.956618e+07
std      9.099981e+08
min     -7.351631e+01
25%      4.000000e+04
50%      7.413192e+04
75%      1.200000e+05
max      2.829740e+10
Name: AdjustedCompensation, dtype: float64

In [10]:
mask = clean_doc_df[clean_doc_df < 0].index
clean_doc_df.drop(mask, inplace=True)
clean_doc_df

22       100000.000
32       172144.440
34       133000.000
61        15000.000
72        43049.736
            ...    
25875     71749.560
25966     12000.000
26012    123553.200
26038    170000.000
26203    200000.000
Name: AdjustedCompensation, Length: 966, dtype: float64

In [11]:
clean_doc_df.describe()

count    9.660000e+02
mean     2.959678e+07
std      9.104690e+08
min      0.000000e+00
25%      4.000000e+04
50%      7.413657e+04
75%      1.200000e+05
max      2.829740e+10
Name: AdjustedCompensation, dtype: float64

In [12]:
clean_bachelors_df.describe()

count    1.107000e+03
mean     6.488710e+04
std      3.069359e+05
min      0.000000e+00
25%      1.126620e+04
50%      3.839940e+04
75%      8.023100e+04
max      9.999999e+06
Name: AdjustedCompensation, dtype: float64

In [13]:
import matplotlib.pyplot as plt
plt.hist(clean_bachelors_df[clean_bachelors_df < 9.999999e+05])
plt.show()

<Figure size 640x480 with 1 Axes>

In [14]:
# drop vals outside +- 1.5*IQR
bach_iqr = stats.iqr(clean_bachelors_df)
print(1.5*bach_iqr)

103447.20000000003


In [15]:
mask = clean_bachelors_df[clean_bachelors_df > (clean_bachelors_df.quantile(0.75) + 1.5*bach_iqr)].index
clean_bachelors_df.drop(mask, inplace=True)
clean_bachelors_df.describe()

count      1067.000000
mean      47539.889016
std       42610.671070
min           0.000000
25%       10521.460000
50%       35667.900000
75%       76122.039000
max      180000.000000
Name: AdjustedCompensation, dtype: float64

In [16]:
mask = clean_bachelors_df[clean_bachelors_df < (clean_bachelors_df.quantile(0.25) - 1.5*bach_iqr)].index
clean_bachelors_df.drop(mask, inplace=True)
clean_bachelors_df.describe()

count      1067.000000
mean      47539.889016
std       42610.671070
min           0.000000
25%       10521.460000
50%       35667.900000
75%       76122.039000
max      180000.000000
Name: AdjustedCompensation, dtype: float64

In [17]:
doc_iqr = stats.iqr(clean_doc_df)
mask = clean_doc_df[clean_doc_df > (clean_doc_df.quantile(0.75) + 1.5*doc_iqr)].index
clean_doc_df.drop(mask, inplace=True)
mask = clean_doc_df[clean_doc_df < (clean_doc_df.quantile(0.25) - 1.5*doc_iqr)].index
clean_doc_df.drop(mask, inplace=True)
clean_doc_df.describe()

count       936.000000
mean      80106.299240
std       54420.477958
min           0.000000
25%       38521.863000
50%       71749.560000
75%      114829.015000
max      240000.000000
Name: AdjustedCompensation, dtype: float64

In [18]:
p = p_value_welch_ttest(clean_bachelors_df, clean_doc_df, two_sided=False)
print(f'Given that our calculated p-value is {round(p,3)}, which is less than our alpha of 0.05')
print(f'We can reject the Null Hypothesis say that there is evidence that education has a positive impact')
print('on salary')

Given that our calculated p-value is 0.0, which is less than our alpha of 0.05
We can reject the Null Hypothesis say that there is evidence that education has a positive impact
on salary


In [19]:
#Your code here

## Wages and Education III

Remember the multiple comparisons problem; rather than continuing on like this, perform an ANOVA test between the various 'FormalEducation' categories and their relation to 'AdjustedCompensation'.

In [20]:
# clean the master's data
mast_iqr = stats.iqr(clean_masters_df)
mask = clean_masters_df[clean_masters_df > (clean_masters_df.quantile(0.75) + 1.5*mast_iqr)].index
clean_masters_df.drop(mask, inplace=True)
mask = clean_masters_df[clean_masters_df < (clean_masters_df.quantile(0.25) - 1.5*mast_iqr)].index
clean_masters_df.drop(mask, inplace=True)
clean_masters_df.describe()


count      1927.000000
mean      58537.417366
std       45085.441382
min           0.000000
25%       20882.400000
50%       50224.692000
75%       87418.060000
max      192810.000000
Name: AdjustedCompensation, dtype: float64

In [21]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [22]:
type(clean_bachelors_df)

pandas.core.series.Series

In [23]:
# convert the series to df's
clean_bachelors_df = clean_bachelors_df.to_frame()
clean_masters_df = clean_masters_df.to_frame()
clean_doc_df = clean_doc_df.to_frame()

# add the degree back into the 3 dataframes
clean_bachelors_df['degree'] = 'bachelors'
clean_masters_df['degree'] = 'masters'
clean_doc_df['degree'] = 'doctorate'

In [24]:
# concatenate all three df's
salary_degree_df = pd.concat([clean_bachelors_df, clean_masters_df, clean_doc_df])
salary_degree_df

,AdjustedCompensation,degree
8,64184.800,bachelors
9,20882.400,bachelors
11,1483.900,bachelors
21,20000.000,bachelors
25,10858.848,bachelors
...,...,...
25875,71749.560,doctorate
25966,12000.000,doctorate
26012,123553.200,doctorate
26038,170000.000,doctorate


In [25]:
formula = 'AdjustedCompensation ~ C(degree)'
lm = ols(formula, salary_degree_df).fit()
table = sm.stats.anova_lm(lm, type=2)
table

,df,sum_sq,mean_sq,F,PR(>F)
C(degree),2.0,5.463055e+11,2.731528e+11,124.446088,3.957086e-53
Residual,3927.0,8.619563e+12,2.194949e+09,NaN,NaN


In [26]:
# the Anova test shows that a Null Hypothesis of Education not having an a positive effect on salary should be rejected
# and the data should be further investigated

## Additional Resources

Here's the original source where the data was taken from:  
    [Kaggle Machine Learning & Data Science Survey 2017](https://www.kaggle.com/kaggle/kaggle-survey-2017)

## Summary

In this lab, you practiced conducting actual hypothesis tests on actual data. From this, you saw how dependent results can be on the initial problem formulation, including preprocessing!